In [2]:
from pynq import Overlay
Proc_RISCV = Overlay("/home/xilinx/pynq/overlays/RISCV_Eirb/design_RISCVEIRB.bit")
Proc_RISCV?

In [3]:
RISCV = Proc_RISCV.My_RISCVEIRB_IP_0 
RISCV?

In [4]:
from pynq import MMIO
BASE_ADDRESS = 0x43C00000
ADDRESS_LENGTH = 0x64000
mmio = MMIO(BASE_ADDRESS, ADDRESS_LENGTH)

In [1]:
from numpy import array

CODE_RAM_SIZE = 32 # This value must be < to 512 (RISCVEIRB Memory sizes)

mem_instruction = array([
    0x00d00513, 
    0x00c00713, 
    0x00700593, 
    0x01c00293, 
    0xff600f93, 
    0x7e600913, 
    0x00058633, 
    0x00c02023, 
    0x00a676b3, 
    0x00d02223, 
    0x00e6e7b3, 
    0x00f02423, 
    0x0057c833, 
    0x01002623, 
    0x40b80833, 
    0x01002823, 
    0x00a838b3, 
    0x01102a23, 
    0x00581333, 
    0x00602c23, 
    0x01f323b3, 
    0x00702e23, 
    0x01233e33, 
    0x03c02023, 
    0x00535eb3, 
    0x03d02223, 
    0x40535f33, 
    0x03e02423, 
    0x000000ef,    
    0x0,
    0x0,
    0x0])

mem_data = array([
    0x11,
    0x7, 
    0xFCFFFE80, 
    0x0f0e0d0c,
    0x13121110, 
    0x17161514, 
    0x1b1a1918, 
    0x1f1e1d1c, 
    0x23222120, 
    0x27262524,
    0x2b2a2928, 
    0x2f2e2d2c, 
    0x76543210, 
    0xfedcba98, 
    0xffffffff, 
    0xffffffff, 
    0xffffffff,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,
    0x0,    
    0x0])

tableauOctets = array([
    0x00, 
    0x00, 
    0x00, 
    0x00])

print("\nRISCV Test\n\r")

#Instruction Memory Write Process
print("\nInstruction Memory Write Process\n\r")
for i in range(0, CODE_RAM_SIZE, 1):
    for j in range (0, 4, 1):
        temp = (j  * 8)
        adresse = (4 * i) + j
        tableauOctets[j]  = (mem_instruction[i]>> temp)%256;
         #print("La valeur de l'instruction est :", hex(tableauOctets[j]), bin(tableauOctets[j]) )
        data_ins = ((0 << 24) + (tableauOctets[j] << 16) + 0b0000000000001010);
         #print("La valeur de la commande binaire :", hex(data_ins), bin(data_ins))
        mmio.write(0x04, adresse)
        mmio.write(0x00, int(data_ins))
         #adresse_rd = mmio.read(0x04)
         #data_ins_rd = mmio.read(0x00)
         #print("La valeur de l'adresse écrite est :", adresse_rd)
         #print("La valeur de la commande écrite est :", hex(data_ins_rd), bin(data_ins_rd))
    print("Octet value Write for current Instruction",hex(mem_instruction[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")

#Data Memory Write Process
print("\nData Memory Write Process\n\r")
for i in range(0, CODE_RAM_SIZE, 1):
    for j in range (0, 4, 1):
        temp = (j  * 8)
        adresse = (4 * i) + j
        tableauOctets[j]  = (mem_data[i]>> temp)%256;
         #print("La valeur de la donnée est :", hex(tableauOctets[j]), bin(tableauOctets[j]) )
        data_val = (tableauOctets[j] << 24) + ((0 << 16) +  + 0b0000000000010100);
         #print("La valeur de la commande binaire :", hex(data_val), bin(data_val))
        mmio.write(0x08, adresse)    
        mmio.write(0x00, int(data_val))
        #adresse_rd = mmio.read(0x04)
        #data_ins_rd = mmio.read(0x00)
        #print("La valeur de l'adresse écrite est :", adresse_rd)        
        #print("La valeur de la commande écrite est :", hex(data_ins_rd), bin(data_ins_rd))
    print("Octet value Write for current Instruction",hex(mem_data[i]),"at address", i,": [",hex(tableauOctets[3]),";",hex(tableauOctets[2]),";",hex(tableauOctets[1]),";",hex(tableauOctets[0]),"]")
    
    
#RISCV-EIRB CPU execution
print("\nExecution of the RISCV-EIRB CPU\n\n\r");
mmio.write(0x0,0b00000000000000000000000000100000) # BOOT <= 1
mmio.write(0x0,0b00000000000000000000000000000000) # BOOT <= 0
for i in range(0, 500, 1):
    #Mode_debug  pour activé le CE : reg0(6)=1 & reg0(0)=1 => (0b00000000000000000000000001000001)
    #Mode_normal pour activé Le CE : reg0(6)=0 & reg0(0)=1 => (0b00000000000000000000000000000001)
    mmio.write(0x0,0b0000000000000000000000001000001)
    mmio.write(0x0,0b0000000000000000000000001000000)
    PC = mmio.read(0x10) # Sig_Adr_Inst_out
    Current_Ins = mmio.read(0x14) # Sig_Val_Out_Inst_out
    NextAdr_Ins = mmio.read(0x18) # Sig_New_Adr_Inst_out
    print("Program Counter (PC) value : ", PC, "(",int(PC/4),"), Current Instruction :", hex(Current_Ins) ,", New Address Instruction :", NextAdr_Ins,"\r")
    Mem_Data_Adr_Value = mmio.read(0x1C)
    Data_in  = mmio.read(0x20) # Sig_Val_In_Data_out
    Data_out = mmio.read(0x24) # Sig_Val_Out_Data_out
    print("Mem Data Address value : ", int(Mem_Data_Adr_Value) ,", Data In : ",hex(Data_in),", Data Out:",hex(Data_out)," \r");
    UAL_op = mmio.read(0x34)
    print("UAL operation: ",bin(UAL_op),"->",int(UAL_op),"\r");
    FSM_value = mmio.read(0x38)
    match int(FSM_value):
        case 0:
            print("Current State Machine is : Init\r")
        case 1:
            print("Current State Machine is : FetchIns\r")
        case 2: 
            print("Current State Machine is : Decode\r")
        case 3: 
            print("Current State Machine is : ExeAddr\r")
        case 4:
            print("Current State Machine is : ExeOp\r")
        case 5:
            print("Current State Machine is : ExeOpimm \r")
        case 6:
            print("Current State Machine is : ExeLoad \r")
        case 7:
            print("Current State Machine is : ExeWrite\r")
        case 8:
            print("Current State Machine is : ExeCtrr\r")
        case 9:
            print("Current State Machine is : ExeJal\r")
        case 10:
            print("Current State Machine is : ExeJal2\r")
        case 11:
            print("Current State Machine is : ExeJalr\r")
        case 12:
            print("Current State Machine is : Undefined\r\n\n")
        case 13:
            print("Current State Machine is : ExeLui\r")
        case 14:
            print("Current State Machine is : ExeAuipc\r")
        case 15:
            print("Current State Machine is : ExeNop\r")
        case _ :
            print("Current State Machine is : Undefined\r\n\n")
    Date_UT_value = mmio.read(0x3C)
    print("Data in UT for Load Instruction",hex(Date_UT_value),",",bin(Date_UT_value),"\r\n")
           
           
#Instruction Memory Read Process  
print("\nInstruction Memory Read Process\n\r")
for i in range(0, CODE_RAM_SIZE, 1):
    for j in range (0, 4, 1):
        mmio.write(0x00,0b00000000000000000000000000000010)
        #val_rd = mmio.read(0x00)
        #print("La valeur écrite est :", hex(val_rd), bin(val_rd))
        adresse = ((4 * i) + j)
        mmio.write(0x04, adresse)
        tableauOctets[j] = mmio.read(0xC)>>16
        #print("La valeur de l'instruction est :", j, hex(tableauOctets[j]), bin(tableauOctets[j]) )
    data_ins_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
    if (data_ins_rd < 0) : 
        data_ins_rd = (data_ins_rd + (1<<32))
    print("La valeur de l'instruction", i," est :", hex(data_ins_rd))#, bin(data_ins_rd))
    
#Data Memory Read Process  
print("\nData Memory Read Process\n\r")
for i in range(0, CODE_RAM_SIZE, 1):
    for j in range (0, 4, 1):
        mmio.write(0x00,0b00000000000000000000000000000100)
        #val_rd = mmio.read(0x00)
        #print("La valeur écrite est :", hex(val_rd), bin(val_rd))
        adresse = ((4 * i) + j)
        mmio.write(0x08, adresse)
        tableauOctets[j] = mmio.read(0xC)>>24
        #print("La valeur de la donnée est :", j, hex(tableauOctets[j]), bin(tableauOctets[j]) )
    data_val_rd = (tableauOctets[3] << 24) + (tableauOctets[2] << 16) + (tableauOctets[1] << 8) + (tableauOctets[0]) 
    if (data_val_rd  < 0) : 
        data_val_rd  = (data_val_rd  + (1<<32))
    print("La valeur de la donnée", i," est :", hex(data_val_rd))#, bin(data_val_rd))   


RISCV Test


Instruction Memory Write Process



NameError: name 'mmio' is not defined

In [66]:
mem_instruction = []

file = open("/home/xilinx/jupyter_notebooks/RISCV/test_op_0_text.hex", "r")
for line in file:
    val = line.strip(',\n')
    mem_instruction.append(val)
file.close()
print( mem_instruction)

[]
0 0x00d00513 ['0x00d00513']
1 0x00c00713 ['0x00d00513', '0x00c00713']
2 0x00700593 ['0x00d00513', '0x00c00713', '0x00700593']
3 0x01c00293 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293']
4 0xff600f93 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93']
5 0x7e600913 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0x7e600913']
6 0x00058633 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0x7e600913', '0x00058633']
7 0x00a676b3 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0x7e600913', '0x00058633', '0x00a676b3']
8 0x00e6e7b3 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0x7e600913', '0x00058633', '0x00a676b3', '0x00e6e7b3']
9 0x0057c833 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0x7e600913', '0x00058633', '0x00a676b3', '0x00e6e7b3', '0x0057c833']
10 0x40b80833 ['0x00d00513', '0x00c00713', '0x00700593', '0x01c00293', '0xff600f93', '0